In [14]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Flatten

MNIST_PATH = os.path.join(os.getcwd(), 'dataset', 'mnist')
MNIST_FILE = os.path.join(MNIST_PATH, 'mnist.npz')

# load dataset from tensorflow
(x_train, y_train), (x_test, y_test) = mnist.load_data(path=MNIST_FILE)

classes = 10
x_train , x_test = x_train/255.0 , x_test/255.0
y_train, y_test = to_categorical(y_train, classes), to_categorical(y_test, classes)

np.savez_compressed(os.path.join(MNIST_PATH, 'x_test'), x_test)
np.savez_compressed(os.path.join(MNIST_PATH, 'y_test'), y_test)
np.savez_compressed(os.path.join(MNIST_PATH, 'x_norm'), x_train[::10])

In [15]:
# nn_model = tf.keras.Sequential([
#     layers.Flatten(input_shape=[28, 28]),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(256, activation='relu'),
#     layers.Dense(units=10, activation='softmax')
# ])
input_shape = x_train.shape[1:]
ann_model = Sequential([
    Flatten(input_shape=[28, 28]),
    Dense(units=784, activation='relu'),
    Dense(units=784*2, activation='relu'),
    Dense(units=classes, activation='softmax')
])

ann_model.compile(optimizer='adam', loss='categorical_crossentropy',
                 metrics=['accuracy'])

ann_model.fit(x_train, y_train, epochs=6, batch_size=256)

Epoch 1/6
235/235 [==============================] - 5s 20ms/step - loss: 0.2263 - accuracy: 0.9320

In [16]:
test_loss,test_acc = ann_model.evaluate(x_test,y_test)
print('Accuracy:',test_acc) # cca 96.67%

313/313 [==============================] - 1s 3ms/step - loss: 0.0766 - accuracy: 0.9792

In [17]:
model_directory = os.path.join(os.getcwd(), 'model',)
if not os.path.exists(model_directory):
    os.makedirs(model_directory)

model_save_path = os.path.join(model_directory, 'mnist_ann_model.h5')
ann_model.save(filepath=str(model_save_path))
print('model saved')

model saved


In [18]:
# https://github.com/NeuromorphicProcessorProject/snn_toolbox/blob/master/examples/mnist_keras_INI.py
from snntoolbox.bin.run import main
from snntoolbox.utils.utils import import_configparser

dt = .1
time_steps_ms = 30
batch_size = 512
test_samples = 10000

config_parser = import_configparser()
config = config_parser.ConfigParser()

config['paths'] = {
    'path-wd': os.path.join(os.getcwd(), 'model'),
    'dataset_path': MNIST_PATH,
    'filename_ann': os.path.join(os.getcwd(), 'model', 'mnist_ann_model')
}

config['tools'] = {
    'evaluate_ann': True,
    'normalize': True
}

config['simulation'] = {
    'simulator': 'INI',
    'duration': time_steps_ms,
    'num_to_test': test_samples,
    'batch_size': batch_size,
    'dt': dt,
    'keras_backed': 'tensorflow'
}

config['cell'] = {
    'tau_refrac': dt,
    'delay': dt
}

config['output'] = {
    'plot_vars': {                  # Various plots (slows down simulation).
        'spiketrains',              # Leave section empty to turn off plots.
        'spikerates',
        'activations',
        'correlation',
        'v_mem',
        'error_t'
    }
}

path = os.getcwd() + '\\config\\'
if not os.path.exists(path):
    os.makedirs(path)

# this for some reason doesnt create the directory
config_filepath = os.getcwd() + '\\config\\mnist-snn-config'
with open(config_filepath, 'w+') as config_file:
    config.write(config_file)

main(config_filepath)

Initializing INI simulator...

Loading scale factors from disk instead of recalculating.
Loading data set from '.npz' files in C:\Users\itznu\PycharmProjects\use-of-spiking-nn\snn-toolbox-examples\dataset\mnist.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 784)               615440    
_________________________________________________________________
dense_19 (Dense)             (None, 1568)              1230880   
_________________________________________________________________
dense_20 (Dense)             (None, 10)                15690     
Total params: 1,862,010
Trainable params: 1,862,010
Non-trainable params: 0
_________________________________________________________________
Evaluating input model on 100

KeyError: '01Dense_784'